In [1]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [10]:
data_path = "data/merged/manhattan_brooklyn_2014_2015.csv"
df = pd.DataFrame.from_csv(data_path, index_col = False)
df.head()

,borough,block,schooldist,council,zipcode,ltdheight,splitzone,bldgclass,landuse,easements,...,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,bbl_pluto,unit_bbl,price_per_sqft
0,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1598000,2014-04-30,2,1920,1,0,1,3.000208e+09,3.000201e+09,57.515117
1,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1598000,2014-04-30,2,1920,1,0,1,3.000208e+09,3.000201e+09,57.515117
2,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,3309312,2014-05-23,2,1920,1,0,1,3.000208e+09,3.000201e+09,119.108552
3,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,3309312,2014-05-23,2,1920,1,0,1,3.000208e+09,3.000201e+09,119.108552
4,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1210000,2014-06-09,2,1920,1,0,1,3.000208e+09,3.000201e+09,43.550245


In [12]:
df.shape

(84948, 51)

In [11]:
no_dups = df.drop_duplicates()
no_dups.shape

(20089, 51)

In [13]:
data_path = "data/merged/finance_manhattan_brooklyn_2014_2015.csv"
finance = pd.DataFrame.from_csv(data_path, index_col = False)
finance.head()

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,block,bbl
0,0,2014-03-21,4,1920,0,90,90,1,3000010050
1,1598000,2014-04-30,2,1920,1,0,1,20,3000201001
2,3309312,2014-05-23,2,1920,1,0,1,20,3000201002
3,1210000,2014-06-09,2,1920,1,0,1,20,3000201003
4,1333907,2014-05-21,2,1920,1,0,1,20,3000201004


In [14]:
finance.shape

(138672, 9)

In [15]:
finance_no_dups = finance.drop_duplicates()
finance_no_dups.shape

(69336, 9)

In [17]:
data_path = "data/merged/pluto_manhattan_brooklyn_2014_2015.csv"
pluto = pd.DataFrame.from_csv(data_path, index_col = False)
pluto.head()

,borough,block,schooldist,council,zipcode,ltdheight,splitzone,bldgclass,landuse,easements,...,condono,xcoord,ycoord,zonemap,latitude,longitude,gross_sqft_pluto,garage,extension,countalter
0,BK,1,13.0,33.0,11201.0,0,0.0,D7,4.0,0,...,0.0,987624.0,195992.0,12d,40.704629,-73.987831,123199.0,0,0,1
1,BK,1,13.0,33.0,11201.0,0,1.0,E9,6.0,0,...,0.0,987838.0,195989.0,12d,40.704621,-73.987059,154400.0,0,0,2
2,BK,7,13.0,33.0,11201.0,0,0.0,G1,10.0,0,...,0.0,987216.0,195803.0,12d,40.704110,-73.989303,9585.0,0,0,1
3,BK,16,13.0,33.0,11201.0,0,0.0,Q0,9.0,0,...,0.0,986856.0,195975.0,12d,40.704582,-73.990601,5000.0,0,0,0
4,BK,18,13.0,33.0,11201.0,0,0.0,D5,3.0,0,...,0.0,987577.0,195811.0,12d,40.704132,-73.988001,211386.0,0,0,1


In [18]:
pluto_no_dups = pluto.drop_duplicates()
pluto_no_dups.shape

(305187, 41)

In [19]:
pluto.shape

(305187, 41)

In [35]:
finance.groupby(['sale_date'])

In [38]:
finance.sort(columns = ['bbl','sale_date'])

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,block,bbl
42290,0,2014-03-01,4,1900,0,0,0,1,1000010010
55548,0,2014-03-01,4,1900,0,0,0,1,1000010010
42291,30150000,2014-10-27,4,0,0,0,1,7,1000071001
55549,30150000,2014-10-27,4,0,0,0,1,7,1000071001
42292,0,2014-10-27,2,0,42,0,42,7,1000071002
55550,0,2014-10-27,2,0,42,0,42,7,1000071002
42293,44250000,2014-03-13,4,1930,0,11,11,8,1000080051
55551,44250000,2014-03-13,4,1930,0,11,11,8,1000080051
42294,10,2014-07-15,4,1987,0,0,0,9,1000090014
55552,10,2014-07-15,4,1987,0,0,0,9,1000090014


In [40]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
new_df = pd.DataFrame.from_csv(data_path, index_col = False)
new_df.head()

,zipcode,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,lottype_7.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2,tax_class_at_time_of_sale_3
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0,0,0,0,1,0,0,1,0,0
1,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0,0,0,0,1,0,0,1,0,0
2,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
4,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0


In [41]:
new_df.shape

(187187, 303)

In [43]:
new_df.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark', 'bbl',
       'xcoord', 'ycoord', 'zonemap', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'sale_price', 'sale_date', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'price_per_sqft', 'subwaydist',
       'botanical_gardens_dist', 'path_stations_dist', 'libraries_dist',
       'colleges_dist', 'sidewalk_cafes_dist', 'day_care_centers_dist',
       'dsny_special_waste_dist', 'link_nyc_centers_dist',
       'beacon_centers_dist', 'out_of_school_youth_centers_dist',
       'runaway_homeless_youth_centers_dist',
       'agency_service_centers_dist',
       'summer_youth_employment_centers_dist',
       'adult_continuing_ed_centers_dist',
  

In [45]:
pluto['bldgclass'] = pluto['bldgclass'].replace(['A1','A2','A3','A4','A5','A6','A7','A8','A9'], ['A' for i in range(9)])

In [48]:
pluto = 

0         D7
1         E9
2         G1
3         Q0
4         D5
5         O6
6         E9
7         C7
8         E9
9         O2
10        F9
11        RM
12        E9
13        E7
14        T2
15        E9
16        J1
17        S3
18        O5
19        J2
20        L9
21        D7
22        RM
23        D7
24        RM
25        RM
26        E9
27        O5
28        S1
29         A
          ..
305157    C7
305158    D4
305159     B
305160    I1
305161    D4
305162    D1
305163    D1
305164    C1
305165    D1
305166    D1
305167    D1
305168    C1
305169    D1
305170     B
305171    C3
305172    D1
305173    R4
305174    D4
305175    D1
305176    C1
305177    C1
305178    D1
305179    D4
305180    D7
305181    D4
305182    D1
305183    C1
305184    D1
305185    D4
305186    C1
Name: bldgclass, dtype: object

In [47]:
pluto['bldgclass'] = pluto['bldgclass'].replace(['B1','B2','B3','B4','B5','B6','B7','B8','B9'], ['B' for i in range(9)])

In [49]:
data_path = "data/merged/manhattan_2010_2010.csv"
pluto = pd.DataFrame.from_csv(data_path, index_col = False)
pluto.head()

,zipcode,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,proxcode_2.0,lottype_0.0,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,1016406.0,0.0,1.0,22.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,15218.0,4260.0,1.0,6.0,3.0,4.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10004.0,0.0,0.0,0.0,10000.0,168748.0,1.0,45.0,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,10004.0,0.0,0.0,0.0,10000.0,168748.0,1.0,45.0,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
pluto.shape

(8730, 161)

In [52]:
pluto.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark', 'bbl',
       'xcoord', 'ycoord', 'zonemap', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'sale_price', 'sale_date', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'price_per_sqft', 'subwaydist',
       'botanical_gardens_dist', 'path_stations_dist', 'libraries_dist',
       'colleges_dist', 'sidewalk_cafes_dist', 'day_care_centers_dist',
       'dsny_special_waste_dist', 'link_nyc_centers_dist',
       'beacon_centers_dist', 'out_of_school_youth_centers_dist',
       'runaway_homeless_youth_centers_dist',
       'agency_service_centers_dist',
       'summer_youth_employment_centers_dist',
       'adult_continuing_ed_centers_dist',
  